# Moving Targets: Basic Workflow

#### Imports

In [ ]:
import target
import plotting
import mast_tap

#### Target Parameters

Setting some parameters up front that will be used elsewhere.

In [ ]:
# 599 Luisa (A906 HF)
# obj_name = '599'  
# id_type = 'smallbody'
# times = {'start':'2010-01-01', 'stop':'2010-01-10', 'step':'1d'}

# Jupiter Barycenter
obj_name = '5'
id_type='majorbody'
times={'start': '2019-04-01', 'stop': '2019-04-10', 'step': '1d'} # WFC3
#times = {'start': '1997-07-10', 'stop': '1997-07-15', 'step': '12h'} # WFPC2

location = None # Geocentric default

#### Get the ephemerides as an astropy table

In [ ]:
eph = target.get_path(obj_name, id_type=id_type, times=times, location=location)

eph

#### Convert this to a space-time coordinate string polygon

See http://www.ivoa.net/documents/STC-S/20130917/WD-STC-S-1.0-20130917.html

In [ ]:
polygon = target.convert_path_to_polygon(eph)

polygon

#### Plotting the polygon

In [ ]:
plotting.polygon_bokeh(polygon, display=True)

#### TAP Query

In [ ]:
start_time = min(eph['datetime_jd']) - 2400000.5
end_time = max(eph['datetime_jd']) - 2400000.5

# If no start/end time supplied, any observation in that footprint would be matched
# results = mast_tap.run_tap_query(polygon, start_time=None, end_time=None, maxrec=100)

results = mast_tap.run_tap_query(polygon, start_time=start_time, end_time=end_time, maxrec=100)

# results

Filter results to only include the ones where the target was in the field of view of the observation. 

In [ ]:
filtered_results = mast_tap.clean_up_results(results, obj_name=obj_name, id_type=id_type, location=location)

filtered_results

#### Display figure

In [ ]:
from bokeh.plotting import figure, output_file, show, output_notebook
from polygon import parse_s_region
from bokeh.models import Arrow, VeeHead
from bokeh.models import HoverTool
from bokeh.palettes import Spectral4

In [ ]:
mast_results = filtered_results
stcs = polygon

p = figure(plot_width=700, x_axis_label="RA (deg)", y_axis_label="Dec (deg)")

# Target path
eph_data = {'eph_x': eph['RA'], 'eph_y': eph['DEC'], 'Date': eph['datetime_str']}
eph_plot1 = p.line(x='eph_x', y='eph_y', source=eph_data, line_width=2, line_color='black')
eph_plot2 = p.circle(x='eph_x', y='eph_y', source=eph_data, fill_color="black", size=12)
p.add_tools(HoverTool(renderers=[eph_plot1, eph_plot2], tooltips=[('Date', "@Date")]))

# Target footprint
patch_xs = parse_s_region(stcs)['ra']
patch_ys = parse_s_region(stcs)['dec']

stcs_data = {'stcs_x': [patch_xs], 'stcs_y': [patch_ys]}
p.patches('stcs_x', 'stcs_y', source=stcs_data, fill_alpha=0., line_color="grey", line_width=0.8, line_dash='dashed')

# Prepare MAST footprints
obsDF = mast_results.to_pandas()
obsDF['coords'] = obsDF.apply(lambda x: parse_s_region(x['s_region']), axis=1)
for col in mast_results.colnames:
    if isinstance(obsDF[col][0], bytes):
        obsDF[col] = obsDF[col].str.decode('utf-8')

# Loop over missions, coloring each separately
mast_plots = []
for mission, color in zip(obsDF['obs_collection'].unique(), Spectral4):
    ind = obsDF['obs_collection'] == mission

    # Add patches with the observation footprings
    patch_xs = [c['ra'] for c in obsDF['coords'][ind]]
    patch_ys = [c['dec'] for c in obsDF['coords'][ind]]

    data = {'x': patch_xs, 'y': patch_ys, 'obs_collection': obsDF['obs_collection'][ind],
            'instrument_name': obsDF['instrument_name'][ind], 'obs_id': obsDF['obs_id'][ind],
            'target_name': obsDF['target_name'][ind], 'proposal_pi': obsDF['proposal_pi'][ind]}
    mast_plots.append(p.patches('x', 'y', source=data, legend=mission,
              fill_color=color, fill_alpha=0.1, line_color="white", line_width=0.5))

# Add hover tooltip for MAST observations
tooltip = [("obs_collection", "@obs_collection"),
           ("instrument_name", "@instrument_name"),
           ("obs_id", "@obs_id"),
           ("target_name", "@target_name"),
           ('proposal_pi', '@proposal_pi')]
p.add_tools(HoverTool(renderers=mast_plots, tooltips=tooltip))

p.x_range.flipped = True
p.legend.click_policy = "hide"

output_notebook()
show(p)